# Notebook de Carga de Datos iniciales

Este notebook esta destinado a la carga inicial de los datos en los distintas base de datos

## Importación de librerias

In [33]:
# Limpia todas las variables existentes
%reset -f

# Habilita autoreload para recargar automáticamente todos los módulos
%load_ext autoreload
%autoreload 2

from pathlib import Path
from db_connections import client,db_neo4j,db_redis
from src import mongo,neo4j,utils,redis
from pymongo.errors import ConnectionFailure
from src.utils import procesar_csv
from datetime import datetime
import os
import pandas as pd


## Conexión con bases de datos

In [34]:
# =====================
# TEST DE CONEXIONES
# =====================

#Conecto con MongoDB
try:
    client.admin.command("ping")  # fuerza conexión al servidor
    print("✅ Conexión a MongoDB verificada.")
except ConnectionFailure as e:
    print(f"❌ Falló la conexión: {type(e).__name__} - {e}")

#Conecto con Neo4j
try:
    db_neo4j.verify_connectivity()
    print("✅ Conexión a Neo4j verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

#Conecto con Redis
try:
    db_redis.ping()
    print("✅ Conexión a Redis verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

✅ Conexión a MongoDB verificada.
✅ Conexión a Neo4j verificada.
✅ Conexión a Redis verificada.


## Limpieza previa

In [35]:
# --- LIMPIAR DATOS EXISTENTES EN TODAS LAS BASES ---

print("\nLimpiando datos existentes...")

# --- MONGO ---
nombre_base = "clase"
client.drop_database(nombre_base)
print("   ✓ Base de datos Mongo eliminada")

# --- REDIS ---
db_redis.flushdb()
print("   ✓ Base de datos Redis limpiada")

# --- NEO4J ---
with db_neo4j.session() as session:
    session.run("MATCH (n) DETACH DELETE n")
print("   ✓ Nodos y relaciones de Neo4j eliminados")

print("\n Todos los datos fueron limpiados correctamente.")



Limpiando datos existentes...
   ✓ Base de datos Mongo eliminada
   ✓ Base de datos Redis limpiada
   ✓ Nodos y relaciones de Neo4j eliminados

 Todos los datos fueron limpiados correctamente.


## Carga de Datos

In [36]:

#-------------------------
# Archivos y colecciones
#-------------------------
nombre_base = "clase"
nombre_colecciones = ["usuarios", "destinos", "hoteles", "reservas", "actividades"]
nombre_archivos = ["usuarios.csv", "destinos.csv", "hoteles.csv", "reservas.csv", "actividades.csv"]

# -------------------------------------------------------------
# PROCESAMIENTO PRINCIPAL
# -------------------------------------------------------------
for nombre_archivo, nombre_coleccion in zip(nombre_archivos, nombre_colecciones):
    df = procesar_csv(nombre_archivo)
    if df is None:
        continue

    mongo.insertar_en_mongo(nombre_base, nombre_coleccion, df)
    redis.insertar_en_redis(nombre_coleccion, df)
    neo4j.crear_nodos_neo4j(nombre_coleccion, df)

    if nombre_coleccion == "reservas":
        neo4j.crear_relaciones_visito(df)

# -------------------------------------------------------------
# RELACIONES ENTRE USUARIOS
# -------------------------------------------------------------
neo4j.crear_relaciones_usuarios()

print("\n🚀 Proceso completado correctamente.")

✅ Se insertaron 60 documentos en 'usuarios'.
✅ Colección usuarios creada e insertada en MongoDB.
✅ Se registran 15 usuarios conectados en Redis.
✅ Nodos de Usuario creados exitosamente en Neo4j.
✅ Se insertaron 37 documentos en 'destinos'.
✅ Colección destinos creada e insertada en MongoDB.
✅ Nodos de Destino creados exitosamente en Neo4j.
✅ Se insertaron 117 documentos en 'hoteles'.
✅ Colección hoteles creada e insertada en MongoDB.
✅ Se insertaron 1685 documentos en 'reservas'.
✅ Colección reservas creada e insertada en MongoDB.
✅ Se insertaron 404 reservas temporales en Redis.


TypeError: crear_relaciones_visito() missing 1 required positional argument: 'nombre_coleccion'

## Verificación de la carga

### MongoDB

##### Cantidad de documentos existentes en cada colección

In [ ]:
for i in range (len(nombre_colecciones)):
    cantidad=mongo.contar_documentos(nombre_base,nombre_colecciones[i])
    print(f"La cantidad de documentos de la coleccion {nombre_colecciones[i]} es {cantidad}")

### Neo4j

In [ ]:
df_nodos = neo4j.consulta(db_neo4j, """
    MATCH (n)
    RETURN labels(n)[0] AS tipo_nodo, COUNT(*) AS cantidad
""")

df_relaciones = neo4j.consulta(db_neo4j, """
    MATCH ()-[r]->()
    RETURN type(r) AS tipo_relacion, COUNT(*) AS cantidad
""")

print("Nodos por tipo:")
print(df_nodos.to_string(index=False))

print("\nRelaciones por tipo:")
print(df_relaciones.to_string(index=False))


### Redis

##### Cantidad y Vista Previa de Usuarios Conectados

In [ ]:
claves = db_redis.keys("usuario:*:sesion")
print(f"Cantidad de usuarios conectados {len(claves)}\n")
print("Se imprimen los primeros 5:")
for clave in claves[:5]:
    usuario_id = clave.split(":")[1]
    estado = db_redis.get(clave)
    tiempo_restante = db_redis.ttl(clave)
    print(f"Usuario {usuario_id} → sesión: {estado} | TTL: {tiempo_restante} segundos")

##### Cantidad y Vista Previa de Reservas en Carrito

In [ ]:
claves = db_redis.keys("reserva_temp:*")
print(f"Cantidad de reservas temporales {len(claves)}\n")
print("Se imprimen las primeras 5:")
for clave in claves[:5]:
    datos = db_redis.hgetall(clave)
    tiempo_restante = db_redis.ttl(clave)
    print(f"{clave}: {datos} | TTL: {tiempo_restante} segundos")

## Generamos algunas búsquedas en caché

In [ ]:
#Borrar redis
#db_redis.flushdb()


consultas = [
    ("hoteles", {"ciudad": "La Plata"}),
    ("destinos", {"tipo": "Playa", "ciudad": "La Rioja"}),
    ("hoteles", {"nombre": "Familia Galiano S.L. Hotel"}),
    ("actividades", {"tipo": "relax"})
]

for tipo, filtro in consultas:
    resultado = redis.obtener_cache(tipo, filtro)
    if resultado is None:
        cursor = mongo.obtener_cursor("clase",nombre_coleccion=tipo,filtro=filtro,proyeccion={"_id":0})
        resultado = list(cursor)
        if resultado:
            redis.guardar_en_cache(tipo,filtro,resultado,ttl=10)   

print("✅ Caché de búsqueda creado")

### Vista del caché de búsqueda para lo guardado

In [ ]:
for tipo, filtro in consultas:
    resultados = redis.obtener_cache(tipo, filtro)
    if resultados:
        for resultado in resultados:
            print(resultado)
    else:
        print(f"No existe información en caché para la búsqueda {filtro} en {tipo}")